In [31]:
import pandas as pd

df = pd.read_excel('./data/fichier forte croissance 2024 avec urls.xlsx')
df.head(3)

,Rang,Nom de l’entreprise,url de l’entreprise,Taux de croissance : Annuel moyen,Taux de croissance : 2019-2022,Chiffre d’affaires : 2022,Chiffre d’affaires : 2019,Salariés : 2022,Salariés : 2019,Salariés : Créations de postes en 2024*,Total 2024,Secteur,Région
0,1,TapNation,https://www.tap-nation.io/,"303,95 %","6.491,36 %",44.324.280 €,672.460 €,33.0,4,25,NaN,"Divertissement, Culture, Multimédia",Île-de-France
1,2,Bathelec,https://www.bathelec.fr/,"257,23 %","4.458,58 %",14.153.329 €,310.477 €,90.0,8,20,NaN,"Construction, Bâtiment",Île-de-France
2,3,Groupe Courtin,https://courtin-promotion.com/,"229,86 %",3.489 %,47.488.703 €,1.323.172 €,60.0,10,50,NaN,Immobilier,Provence-Alpes-Côte d’Azur-Corse


In [36]:
df.columns = ['Rang', 'Nom_de_l_entreprise', 'url_de_l_entreprise',
       'Taux_de_croissance_Annuel_moyen', 'Taux_de_croissance_2019_2022',
       'Chiffre_d_affaires_2022', 'Chiffre_d_affaires_2019',
       'Salariés_2022', 'Salariés_2019',
       'Salariés_Créations_de_postes_en_2024', 'Total_2024', 'Secteur',
       'Région']

In [40]:
df.head(3)

,Rang,Nom_de_l_entreprise,url_de_l_entreprise,Taux_de_croissance_Annuel_moyen,Taux_de_croissance_2019_2022,Chiffre_d_affaires_2022,Chiffre_d_affaires_2019,Salariés_2022,Salariés_2019,Salariés_Créations_de_postes_en_2024,Total_2024,Secteur,Région
0,1,TapNation,https://www.tap-nation.io/,"303,95 %","6.491,36 %",44.324.280 €,672.460 €,33.0,4,25,NaN,"Divertissement, Culture, Multimédia",Île-de-France
1,2,Bathelec,https://www.bathelec.fr/,"257,23 %","4.458,58 %",14.153.329 €,310.477 €,90.0,8,20,NaN,"Construction, Bâtiment",Île-de-France
2,3,Groupe Courtin,https://courtin-promotion.com/,"229,86 %",3.489 %,47.488.703 €,1.323.172 €,60.0,10,50,NaN,Immobilier,Provence-Alpes-Côte d’Azur-Corse


In [44]:
df.columns, len(df.columns)

(Index(['Rang', 'Nom_de_l_entreprise', 'url_de_l_entreprise',
        'Taux_de_croissance_Annuel_moyen', 'Taux_de_croissance_2019_2022',
        'Chiffre_d_affaires_2022', 'Chiffre_d_affaires_2019', 'Salariés_2022',
        'Salariés_2019', 'Salariés_Créations_de_postes_en_2024', 'Total_2024',
        'Secteur', 'Région'],
       dtype='object'),
 13)

In [48]:
df.to_csv('results/entreprises.csv', index=False)

In [3]:
!docker-compose up -d

time="2024-10-11T11:08:56+01:00" level=warning msg="C:\\Users\\Windows\\Downloads\\Projects\\Hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024\\docker-compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 Network hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024_default  Creating
 Network hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024_default  Created
 Container selenium-hub-hajy8_letudiant-project  Creating
 Container selenium-hub-hajy8_letudiant-project  Created
 Container hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024-chrome_1_hajy8_letudiant_project-1  Creating
 Container hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024-chrome_1_hajy8_letudiant_project-1  Created
 Container selenium-hub-hajy8_letudiant-project  Starting
 Container selenium-hub-hajy8_letudiant-project  Started
 Container hajy8_contact_scraping_fichier_entreprises_forte_cr

In [1]:
from ItemScraping import ItemScraping
from ItemStorage import ItemStorage
from Item import Item

from contact_links_classification.ContactLinkModel import ContactLinkModel
from OpenAI_API import ContactOpenAIScraping, FoundersOpenAIClassification
from pageProcessing import PageProcessing
from sentenceProcessing import SentenceProcessing

import pandas as pd
import time
import csv
import codecs
import threading

file_path = 'results/entreprises'

def difference_dic(dic1, dic2):
    # Extraire les valeurs de 'Nom_de_l_entreprise' de dic2 dans un set pour une recherche rapide
    entreprises_dic2 = {d['Nom_de_l_entreprise'] for d in dic2}
    
    # Retourner les dictionnaires dans dic1 qui ne sont pas dans dic2 (selon 'Nom_de_l_entreprise')
    return [d for d in dic1 if d['Nom_de_l_entreprise'] not in entreprises_dic2]
    
def csv_to_dict_list(file_path):
    # Utiliser pandas pour lire le CSV avec un encodage automatique
    df = pd.read_csv(file_path)  # ou 'latin1'
    list_of_dicts = df.to_dict(orient='records')
    # Remplacer \xa0 par un espace vide pour chaque valeur dans le dictionnaire
    for item in list_of_dicts:
        for key, value in item.items():
            if isinstance(value, str):  # Vérifie si la valeur est une chaîne de caractères
                item[key] = value.replace('\xa0', ' ')  # Supprime \xa0
                
    return list_of_dicts
    
# Fonction pour chaque thread qui pop un élément de la liste et traite le dictionnaire
def worker(list_of_dicts, lock_list_of_dicts, results, lock_results, for_task, lock_selenium_grid, lock_print):

    if for_task == 'extract_contact_info' :
        contact_link_classifier = ContactLinkModel()
        contact_link_classifier.load_from_local(model_path='./contact_links_classification/Models/model_0/model_contact_40_maxlen_10_epochs')
        contactOpenAIScraping = ContactOpenAIScraping()
        pageProcessing = PageProcessing()
        sentenceProcessing = SentenceProcessing(max_words_before_phone_number_or_email=30)
        
    if for_task == 'extract_Linkedin_url_from_google' :
        with lock_selenium_grid:
            itemScraping = ItemScraping(with_selenium_grid = True)
        itemScraping.get_google_page()

    if for_task == 'extract_LinkedIn_profiles_of_company_founders' :
        with lock_selenium_grid:
            itemScraping = ItemScraping(with_selenium_grid = False)
        itemScraping.get_linkedin_authentication()
        
    if for_task == 'Founder_Profiles_Classification_with_OpenAi':
        foundersOpenAIClassification = FoundersOpenAIClassification()
        with lock_selenium_grid:
            itemScraping = ItemScraping(foundersOpenAIClassification=foundersOpenAIClassification, with_selenium_grid = False, lock_print=lock_print)
            itemScraping.driver.quit()
        
    while True:
        # Utilisation de 'with lock_list_of_dicts' pour synchroniser l'accès à la liste
        with lock_list_of_dicts:
            if list_of_dicts:
                item_dic = list_of_dicts.pop(0)  # Récupérer un dictionnaire de la liste
            else:
                break  # Si la liste est vide, sortir de la boucle

        # Traiter le dictionnaire (en dehors du verrou pour ne pas bloquer les autres threads)
        item = Item()
        item.init_from_dic(item_dic)
  
        if for_task == 'extract_contact_info' :
            with lock_selenium_grid:
                itemScraping = ItemScraping(url=item.url_de_l_entreprise,  # a modifier
                                            item=item,
                                            contact_link_classifier=contact_link_classifier, contactOpenAIScraping=contactOpenAIScraping,
                                            pageProcessing=pageProcessing, sentenceProcessing=sentenceProcessing)
            result = itemScraping.get_all_contact_page_links()
            
        if for_task == 'extract_Linkedin_url_from_google' :
            result = itemScraping.get_linkedin_url_from_company_name(item)
            if not  result['status']:
                with lock_selenium_grid:
                    itemScraping = ItemScraping()
                itemScraping.get_google_page()

        if for_task == 'extract_LinkedIn_profiles_of_company_founders':
            result = itemScraping.get_LinkedIn_profiles_of_company_founders(item)
            
            if not  result['status']:
                with lock_selenium_grid:
                    itemScraping = ItemScraping(with_selenium_grid = False)
                itemScraping.get_linkedin_authentication()
                
        if for_task == 'Founder_Profiles_Classification_with_OpenAi' :
            result = itemScraping.get_Founder_Profiles_using_OpenAi(item)
            
            
        # print(result['data'])

        #Sauvegarder le résultat (en utilisant le verrou pour protéger l'accès à la liste des résultats)
        if result['status']:
            with lock_results:
                ItemStorage(f'{file_path}_with_contact_inf_55', value=result['data'])
                results.append(result['data'])


# Fonction principale pour lancer les threads
def process_with_threads(list_of_dicts_items, num_threads=10, for_task='extract_contact_info'):
    results = []
    # Deux verrous, un pour la liste de dictionnaires et un pour les résultats
    lock_list_of_dicts = threading.Lock()
    lock_results = threading.Lock()
    lock_selenium_grid = threading.Lock()
    lock_print = threading.Lock()
    
    # Créer et lancer les threads
    threads = []
    for _ in range(num_threads):
        t = threading.Thread(target=worker, args=(list_of_dicts_items, lock_list_of_dicts, results, lock_results, for_task, lock_selenium_grid, lock_print))
        t.start()
        threads.append(t)

    # Attendre que tous les threads aient fini
    for t in threads:
        t.join()

    return results

def main():
    complet_file_path = f'{file_path}.csv'
    complet_file_path = 'results/exceptions.csv'
    complet_file_path = f'{file_path}_with_contact_inf_3.csv'

    list_of_dicts_items = csv_to_dict_list(complet_file_path)
    print(len(list_of_dicts_items))
    
    results = process_with_threads(list_of_dicts_items[:20], num_threads=10, for_task = 'Founder_Profiles_Classification_with_OpenAi')
    
    print(f'*'*150)

    ItemStorage(f'{file_path}_with_contact_inf_5', value=results)
    
if __name__ == "__main__":
    
    main()
    # # Lire le fichier CSV
    df = pd.read_csv(f'{file_path}_with_contact_inf_5.csv')
    df_sorted = df.sort_values(by='Rang')
    # # Enregistrer dans un fichier Excel
    df_sorted.to_excel(f'{file_path}_with_contact_inf_5.xlsx', index=False, engine='openpyxl')

500
1 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  TapNation   @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
2 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  Bathelec   @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Founder profiles:
3 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  Groupe Courtin   @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Founder profiles:
4 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  Valumen Groupe   @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Founder profiles:
Founder profiles:
5 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  YSO Corp   @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
6 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  Adrenalead   @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Founder profiles:
7 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  Voltaïca   @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Founder profiles:
8 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  Educ-Up

In [5]:
print(f"{'@'*50} aaa {'@'*50}")

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ aaa @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


In [10]:
import pandas as pd
file_path = 'results/entreprises'
# # Lire le fichier CSV
df = pd.read_csv(f'{file_path}_with_contact_inf_44.csv')
df_sorted = df.sort_values(by='Rang')
# # Enregistrer dans un fichier Excel
df_sorted.to_excel(f'{file_path}_with_contact_inf_4.xlsx', index=False, engine='openpyxl')
df_sorted.to_csv(f'{file_path}_with_contact_inf_4.csv', index=False)

In [1]:
def difference_dic(dic1, dic2):
    # Extraire les valeurs de 'Nom_de_l_entreprise' de dic2 dans un set pour une recherche rapide
    entreprises_dic2 = {d['Nom_de_l_entreprise'] for d in dic2}
    
    # Retourner les dictionnaires dans dic1 qui ne sont pas dans dic2 (selon 'Nom_de_l_entreprise')
    return [d for d in dic1 if d['Nom_de_l_entreprise'] not in entreprises_dic2]

# Exemple d'utilisation
dic1 = [
    {'Nom_de_l_entreprise': 'Entreprise A', 'Adresse': 'Adresse A'},
    {'Nom_de_l_entreprise': 'Entreprise B', 'Adresse': 'Adresse B'},
    {'Nom_de_l_entreprise': 'Entreprise C', 'Adresse': 'Adresse C'}
]

dic2 = [
    {'Nom_de_l_entreprise': 'Entreprise B', 'Adresse': 'Adresse X'},
    {'Nom_de_l_entreprise': 'Entreprise D', 'Adresse': 'Adresse Y'}
]

resultat = difference_dic(dic1, dic2)
print(resultat)


[{'Nom_de_l_entreprise': 'Entreprise A', 'Adresse': 'Adresse A'}, {'Nom_de_l_entreprise': 'Entreprise C', 'Adresse': 'Adresse C'}]


In [47]:
import pandas as pd
df = pd.read_csv(f'results/entreprises_with_contact_inf_1.csv')
df_sorted = df.sort_values(by='Rang')
df_sorted.to_csv(f'results/entreprises_with_contact_inf_1.csv', encoding='utf-8-sig', index=False)

df = pd.read_csv(f'results/entreprises_with_contact_inf_1.csv')
# # Enregistrer dans un fichier Excel
df.to_excel(f'results/entreprises_with_contact_inf_1.xlsx',  index=False, engine='openpyxl')

In [1]:
import pandas as pd
df = pd.read_excel(f'results/entreprises_with_contact_inf_1.xlsx')
df.to_csv(f'results/entreprises_with_contact_inf_1.csv', encoding='utf-8-sig', index=False)